<a href="https://colab.research.google.com/github/1804054Miraz/Paper-Publishing/blob/main/ML_apply_in_on_Gamma_and_KatzFD_Band_with_full_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import numpy as np
import os
import pandas as pd
import statistics as st
import scipy.stats as scst
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
from scipy.stats import normaltest
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time
from sklearn.ensemble import BaggingClassifier

In [2]:
path = '/content/drive/MyDrive/Dataset/Features/merged_gamma_a_c.csv'
merged_gamma_a_c = pd.read_csv(path)
# merged_gamma_a_c=merged_gamma_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)
merged_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,label
0,5.867765,16.136485,14.419578,107.004679,4.771000,10.412779,2.264702,4.967259,4.718617,2.227520,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,4.788753,11.568614,14.381443,75.048537,2.176075,8.500067,1.558222,4.906503,3.750594,1.237990,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,6.297802,29.544646,13.712807,97.931325,4.131878,13.930406,1.443750,4.954165,4.627834,2.631432,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,4.348968,9.195677,10.247385,62.184384,2.506088,6.808748,1.321086,3.921911,3.681952,1.816489,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,7.698942,15.086431,20.116347,61.128746,6.571280,8.982171,3.578223,4.040258,6.796313,2.204725,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,2.437103,2.691105,9.169849,2.129555,1.467451,1.610917,0.581611,2.188392,2.580653,1.218395,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,0.897495,1.245264,3.604244,3.566347,1.042233,1.365339,0.834950,1.492575,0.902565,1.093884,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,1.251944,1.448347,6.375324,4.212090,0.632365,0.651843,0.297628,0.621490,0.765336,1.009508,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal
7620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,normal


In [4]:
merged_gamma_a_c = merged_gamma_a_c.replace(0, merged_gamma_a_c.mean(numeric_only=True))
# merged_gamma_a_c = merged_gamma_a_c.drop(7620)
# merged_gamma_a_c = merged_gamma_a_c.reset_index(drop=True)
merged_gamma_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,label
0,5.867765,16.136485,14.419578,107.004679,4.771000,10.412779,2.264702,4.967259,4.718617,2.227520,...,31.407003,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,addicted
1,4.788753,11.568614,14.381443,75.048537,2.176075,8.500067,1.558222,4.906503,3.750594,1.237990,...,31.837468,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,addicted
2,6.297802,29.544646,13.712807,97.931325,4.131878,13.930406,1.443750,4.954165,4.627834,2.631432,...,39.507001,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,addicted
3,4.348968,9.195677,10.247385,62.184384,2.506088,6.808748,1.321086,3.921911,3.681952,1.816489,...,30.478540,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,addicted
4,7.698942,15.086431,20.116347,61.128746,6.571280,8.982171,3.578223,4.040258,6.796313,2.204725,...,55.854653,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,2.437103,2.691105,9.169849,2.129555,1.467451,1.610917,0.581611,2.188392,2.580653,1.218395,...,5.024580,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,normal
7618,0.897495,1.245264,3.604244,3.566347,1.042233,1.365339,0.834950,1.492575,0.902565,1.093884,...,3.040292,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,normal
7619,1.251944,1.448347,6.375324,4.212090,0.632365,0.651843,0.297628,0.621490,0.765336,1.009508,...,2.070562,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,normal
7620,11.282638,10.411617,9.804037,11.176228,3.779628,3.187675,1.696841,2.772710,2.596294,5.984265,...,7.213328,0.661268,2.635775,5.232391,1.594932,1.549560,0.637978,6.253633,7.101868,normal


In [5]:
X = merged_gamma_a_c.iloc[:, :-1]
y = merged_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Support Vector Machine (SVM)

svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
svm_precision = precision_score(y_encoded, svm_predictions)
svm_recall = recall_score(y_encoded, svm_predictions)
svm_f1 = f1_score(y_encoded, svm_predictions)


print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1 Score:", svm_f1)
# print("Processing Time:", svm_processing_time)

# XGBoost

xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
xgb_precision = precision_score(y_encoded, xgb_predictions)
xgb_recall = recall_score(y_encoded, xgb_predictions)
xgb_f1 = f1_score(y_encoded, xgb_predictions)


print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
print("Precision:", xgb_precision)
print("Recall:", xgb_recall)
print("F1 Score:", xgb_f1)


# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
rf_precision = precision_score(y_encoded, rf_predictions)
rf_recall = recall_score(y_encoded, rf_predictions)
rf_f1 = f1_score(y_encoded, rf_predictions)

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1 Score:", rf_f1)


# Ensemble Subspace of k-Nearest Neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
print("Precision:", ensemble_knn_precision)
print("Recall:", ensemble_knn_recall)
print("F1 Score:", ensemble_knn_f1)
# print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
print("LSTM Model Precision:", np.mean(precision_scores))
print("LSTM Model Recall:", np.mean(recall_scores))
print("LSTM Model F1 Score:", np.mean(f1_scores))



SVM Metrics:
Accuracy: 0.8454473891367095
Precision: 0.8692847124824684
Recall: 0.8131723956966675
F1 Score: 0.84029284164859

XGBoost Metrics:
Accuracy: 0.9553922854893728
Precision: 0.9523064894448788
Recall: 0.9588034636578326
F1 Score: 0.9555439330543932

Random Forest Metrics:
Accuracy: 0.9286276567829966
Precision: 0.9241755388210854
Recall: 0.9338756231960116
F1 Score: 0.9290002610284521


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.9441091577013907
Precision: 0.9401820546163849
Recall: 0.9485699291524534
F1 Score: 0.9443573667711599
24/24 [==============================] - 0s 1ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.9178675486665085
LSTM Model Precision: 0.9083745695846716
LSTM Model Recall: 0.9323798571681351
LSTM Model F1 Score: 0.9193176057750454


In [6]:
pip install xgboost lightgbm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.4 MB/s eta 0:00:00


In [7]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# LightGBM
lgb_classifier = lgb.LGBMClassifier()
lgb_predictions = cross_val_predict(lgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
lgb_accuracy = accuracy_score(y_encoded, lgb_predictions)
lgb_precision = precision_score(y_encoded, lgb_predictions)
lgb_recall = recall_score(y_encoded, lgb_predictions)
lgb_f1 = f1_score(y_encoded, lgb_predictions)

print("\nLightGBM Metrics:")
print("Accuracy:", lgb_accuracy)
print("Precision:", lgb_precision)
print("Recall:", lgb_recall)
print("F1 Score:", lgb_f1)

from catboost import CatBoostClassifier

# CatBoost
catboost_classifier = CatBoostClassifier(verbose=0)
catboost_predictions = cross_val_predict(catboost_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
catboost_accuracy = accuracy_score(y_encoded, catboost_predictions)
catboost_precision = precision_score(y_encoded, catboost_predictions)
catboost_recall = recall_score(y_encoded, catboost_predictions)
catboost_f1 = f1_score(y_encoded, catboost_predictions)

print("\nCatBoost Metrics:")
print("Accuracy:", catboost_accuracy)
print("Precision:", catboost_precision)
print("Recall:", catboost_recall)
print("F1 Score:", catboost_f1)



AdaBoost Metrics:
Accuracy: 0.7883757543951718
Precision: 0.7886029411764706
Recall: 0.7879821569141957
F1 Score: 0.7882924268276676

GBM Metrics:
Accuracy: 0.8837575439517187
Precision: 0.8766417718259079
Recall: 0.8932038834951457
F1 Score: 0.8848453340265142
[LightGBM] [Info] Number of positive: 3439, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005455 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501385 -> initscore=0.005540
[LightGBM] [Info] Start training from score 0.005540
[LightGBM] [Info] Number of positive: 3446, number of negative: 3413
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info

In [12]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Prepare the data
X = merged_gamma_a_c.iloc[:, :-1]
y = merged_gamma_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
print("Precision:", np.mean(voting_scores['test_precision']))
print("Recall:", np.mean(voting_scores['test_recall']))
print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3439, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005702 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501385 -> initscore=0.005540
[LightGBM] [Info] Start training from score 0.005540
[LightGBM] [Info] Number of positive: 3446, number of negative: 3413
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005630 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502406 -> initscore=0.009622
[LightGBM] [Info] Start training from score 0.009622
[LightGBM] [Info] Nu

In [13]:
path = '/content/drive/MyDrive/Dataset/Features/merged_gamma_a_c.csv'
merged_gamma_a_c = pd.read_csv(path)
# merged_gamma_a_c=merged_gamma_a_c.drop(columns=['X_PS','Y_PS','nd_PS'], axis=False)

path = '/content/drive/MyDrive/Dataset/Features/Katz_Fd_Features_a_c.csv'
Katz_Fd_Features_a_c = pd.read_csv(path)

merged_gamma_katz_a_c = pd.concat([merged_gamma_a_c,Katz_Fd_Features_a_c], axis=1)
# merged_beta_gamma_a_c
merged_gamma_katz_a_c.drop('label', axis=1, inplace=True)
merged_gamma_katz_a_c['label']='normal'
merged_gamma_katz_a_c.loc[0:3810, 'label'] = 'addicted'
merged_gamma_katz_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,FP2,label
0,5.867765,16.136485,14.419578,107.004679,4.771000,10.412779,2.264702,4.967259,4.718617,2.227520,...,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,1.655864,addicted
1,4.788753,11.568614,14.381443,75.048537,2.176075,8.500067,1.558222,4.906503,3.750594,1.237990,...,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,1.367933,addicted
2,6.297802,29.544646,13.712807,97.931325,4.131878,13.930406,1.443750,4.954165,4.627834,2.631432,...,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,1.584968,addicted
3,4.348968,9.195677,10.247385,62.184384,2.506088,6.808748,1.321086,3.921911,3.681952,1.816489,...,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,1.282758,addicted
4,7.698942,15.086431,20.116347,61.128746,6.571280,8.982171,3.578223,4.040258,6.796313,2.204725,...,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,1.461674,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,2.437103,2.691105,9.169849,2.129555,1.467451,1.610917,0.581611,2.188392,2.580653,1.218395,...,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,1.538744,normal
7618,0.897495,1.245264,3.604244,3.566347,1.042233,1.365339,0.834950,1.492575,0.902565,1.093884,...,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,1.716444,normal
7619,1.251944,1.448347,6.375324,4.212090,0.632365,0.651843,0.297628,0.621490,0.765336,1.009508,...,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,1.556286,normal
7620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,normal


In [15]:
merged_gamma_katz_a_c = merged_gamma_katz_a_c.replace(0, merged_gamma_katz_a_c.mean(numeric_only=True))
# merged_gamma_katz_a_c = merged_gamma_katz_a_c.drop(7620)
# merged_gamma_katz_a_c = merged_gamma_katz_a_c.reset_index(drop=True)
merged_gamma_katz_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,FP2,label
0,5.867765,16.136485,14.419578,107.004679,4.771000,10.412779,2.264702,4.967259,4.718617,2.227520,...,0.639929,3.175360,7.257440,1.171091,1.727645,0.303052,5.626485,6.090333,1.655864,addicted
1,4.788753,11.568614,14.381443,75.048537,2.176075,8.500067,1.558222,4.906503,3.750594,1.237990,...,0.481474,1.360249,3.002968,0.452312,0.873387,0.169722,4.693551,3.472028,1.367933,addicted
2,6.297802,29.544646,13.712807,97.931325,4.131878,13.930406,1.443750,4.954165,4.627834,2.631432,...,0.403271,1.391440,2.908869,0.790318,1.077771,0.227142,6.266331,3.612442,1.584968,addicted
3,4.348968,9.195677,10.247385,62.184384,2.506088,6.808748,1.321086,3.921911,3.681952,1.816489,...,0.450695,1.121842,2.162650,0.753562,0.999099,0.180620,4.211706,6.950914,1.282758,addicted
4,7.698942,15.086431,20.116347,61.128746,6.571280,8.982171,3.578223,4.040258,6.796313,2.204725,...,1.011451,3.663678,5.646425,1.918202,2.340282,0.437771,7.413797,7.107321,1.461674,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,2.437103,2.691105,9.169849,2.129555,1.467451,1.610917,0.581611,2.188392,2.580653,1.218395,...,0.227139,3.721454,4.377724,1.713244,1.729774,0.453551,5.866098,3.061931,1.538744,normal
7618,0.897495,1.245264,3.604244,3.566347,1.042233,1.365339,0.834950,1.492575,0.902565,1.093884,...,0.398802,3.180998,2.880482,2.019003,1.853624,0.785453,2.408728,3.512361,1.716444,normal
7619,1.251944,1.448347,6.375324,4.212090,0.632365,0.651843,0.297628,0.621490,0.765336,1.009508,...,0.132006,1.298873,1.126842,0.855509,0.550932,0.247462,1.363607,3.663259,1.556286,normal
7620,11.282638,10.411617,9.804037,11.176228,3.779628,3.187675,1.696841,2.772710,2.596294,5.984265,...,0.661268,2.635775,5.232391,1.594932,1.549560,0.637978,6.253633,7.101868,1.614143,normal


In [16]:
# Assuming 'new_dataframe' is your DataFrame
X = merged_gamma_katz_a_c.iloc[:, :-1]
y = merged_gamma_katz_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Support Vector Machine (SVM)
start_time = time.time()
svm_classifier = make_pipeline(StandardScaler(), SVC())
svm_predictions = cross_val_predict(svm_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
svm_accuracy = accuracy_score(y_encoded, svm_predictions)
svm_precision = precision_score(y_encoded, svm_predictions)
svm_recall = recall_score(y_encoded, svm_predictions)
svm_f1 = f1_score(y_encoded, svm_predictions)
svm_processing_time = time.time() - start_time

print("SVM Metrics:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1 Score:", svm_f1)
print("Processing Time:", svm_processing_time)

# XGBoost
start_time = time.time()
xgb_classifier = XGBClassifier()
xgb_predictions = cross_val_predict(xgb_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
xgb_accuracy = accuracy_score(y_encoded, xgb_predictions)
xgb_precision = precision_score(y_encoded, xgb_predictions)
xgb_recall = recall_score(y_encoded, xgb_predictions)
xgb_f1 = f1_score(y_encoded, xgb_predictions)
xgb_processing_time = time.time() - start_time

print("\nXGBoost Metrics:")
print("Accuracy:", xgb_accuracy)
print("Precision:", xgb_precision)
print("Recall:", xgb_recall)
print("F1 Score:", xgb_f1)
print("Processing Time:", xgb_processing_time)


# Random Forest
start_time = time.time()
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = cross_val_predict(rf_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
rf_accuracy = accuracy_score(y_encoded, rf_predictions)
rf_precision = precision_score(y_encoded, rf_predictions)
rf_recall = recall_score(y_encoded, rf_predictions)
rf_f1 = f1_score(y_encoded, rf_predictions)
rf_processing_time = time.time() - start_time

print("\nRandom Forest Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1 Score:", rf_f1)
print("Processing Time:", rf_processing_time)


# Ensemble Subspace of k-Nearest Neighbors
start_time_ensemble_knn = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=5)
ensemble_knn_classifier = BaggingClassifier(base_estimator=knn_classifier, n_estimators=10, random_state=42)

ensemble_knn_scores = cross_val_predict(ensemble_knn_classifier, X, y_encoded, cv=KFold(n_splits=10, shuffle=True, random_state=42))
ensemble_knn_accuracy = accuracy_score(y_encoded, ensemble_knn_scores)
ensemble_knn_precision = precision_score(y_encoded, ensemble_knn_scores)
ensemble_knn_recall = recall_score(y_encoded, ensemble_knn_scores)
ensemble_knn_f1 = f1_score(y_encoded, ensemble_knn_scores)
ensemble_knn_processing_time = time.time() - start_time_ensemble_knn

print("\nEnsemble Subspace of k-Nearest Neighbors Metrics:")
print("Accuracy:", ensemble_knn_accuracy)
print("Precision:", ensemble_knn_precision)
print("Recall:", ensemble_knn_recall)
print("F1 Score:", ensemble_knn_f1)
print("Processing Time:", ensemble_knn_processing_time)

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Convert DataFrame to numpy array
X_np = X.values
X_np = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

# 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for train_idx, test_idx in kfold.split(X_np):
    X_train, X_test = X_np[train_idx], X_np[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    # Create and fit LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = create_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predictions
    y_pred_probs = lstm_model.predict(X_test)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

print("\nLSTM Metrics:")
print("LSTM Model Accuracy:", np.mean(accuracy_scores))
print("LSTM Model Precision:", np.mean(precision_scores))
print("LSTM Model Recall:", np.mean(recall_scores))
print("LSTM Model F1 Score:", np.mean(f1_scores))


SVM Metrics:
Accuracy: 0.84912096562582
Precision: 0.8608624898291294
Recall: 0.8328522697454737
F1 Score: 0.8466257668711656
Processing Time: 29.815613985061646

XGBoost Metrics:
Accuracy: 0.953686696405143
Precision: 0.9504950495049505
Recall: 0.9572290737339281
F1 Score: 0.9538501764936592
Processing Time: 18.469693422317505

Random Forest Metrics:
Accuracy: 0.9296772500655995
Precision: 0.9230173081890984
Recall: 0.9375491996851221
F1 Score: 0.9302265035147098
Processing Time: 71.9840190410614


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en


Ensemble Subspace of k-Nearest Neighbors Metrics:
Accuracy: 0.944240356861716
Precision: 0.9401976079043162
Recall: 0.9488323274731042
F1 Score: 0.944495233120021
Processing Time: 4.4972145557403564
24/24 [==============================] - 0s 1ms/step

LSTM Metrics:
LSTM Model Accuracy: 0.923509389307988
LSTM Model Precision: 0.9303110631990787
LSTM Model Recall: 0.9172833324946158
LSTM Model F1 Score: 0.9228852084733411


In [18]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
print("Precision:", np.mean(voting_scores['test_precision']))
print("Recall:", np.mean(voting_scores['test_recall']))
print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3439, number of negative: 3420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005729 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16575
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501385 -> initscore=0.005540
[LightGBM] [Info] Start training from score 0.005540
[LightGBM] [Info] Number of positive: 3446, number of negative: 3413
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16575
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502406 -> initscore=0.009622
[LightGBM] [Info] Start training from score 0.009622
[LightGBM] [Info] Nu

In [20]:
path = '/content/drive/MyDrive/Dataset/Features/merged_beta_a_c.csv'
merged_beta_a_c = pd.read_csv(path)
merged_beta_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,85.385627,3.036687,24.795219,43.270243,12.162779,12.929786,2.030020,48.186443,62.769034,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,37.463505,2.172771,17.780453,35.359290,11.068525,9.404205,1.722296,30.269310,59.009956,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,65.876952,2.695904,20.281652,38.884344,9.680058,12.907316,2.183548,34.001842,31.580880,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,64.519703,4.775083,43.981373,72.999857,27.102261,20.996296,3.607103,39.967449,45.141392,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,96.734399,3.988225,29.985422,53.775181,13.792507,20.360930,2.889787,51.401620,46.543616,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,195.318946,11.285102,173.717064,174.860439,88.949160,86.029325,26.312015,214.825811,91.225219,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,90.841723,5.113646,49.058404,54.869916,41.159297,29.461572,11.418493,61.084965,53.495500,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,90.035788,14.752258,92.281538,105.084474,51.557043,60.772363,21.107574,126.136996,112.691773,normal
7620,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,normal


In [21]:
merged_beta_a_c = merged_beta_a_c.replace(0, merged_beta_a_c.mean(numeric_only=True))
# merged_gamma_a_c = merged_gamma_a_c.drop(7620)
# merged_gamma_a_c = merged_gamma_a_c.reset_index(drop=True)
merged_beta_a_c

,FP1_PS,FP2_PS,F7_PS,F8_PS,AF1_PS,AF2_PS,FZ_PS,F4_PS,F3_PS,FC6_PS,...,PO8_PS,FCZ_PS,POZ_PS,OZ_PS,P2_PS,P1_PS,CPZ_PS,nd_PS,Y_PS,label
0,49.125112,75.401561,55.085147,236.773934,24.574587,34.914910,9.395342,15.433209,21.208462,9.854879,...,85.385627,3.036687,24.795219,43.270243,12.162779,12.929786,2.030020,48.186443,62.769034,addicted
1,31.662543,140.818732,107.251892,299.075377,22.347470,53.286704,9.112230,29.795814,17.161432,20.283656,...,37.463505,2.172771,17.780453,35.359290,11.068525,9.404205,1.722296,30.269310,59.009956,addicted
2,33.981550,119.618734,69.705635,404.815675,24.381613,75.511201,9.212180,28.717643,24.864757,23.216713,...,65.876952,2.695904,20.281652,38.884344,9.680058,12.907316,2.183548,34.001842,31.580880,addicted
3,39.844284,101.587017,85.574482,491.630726,27.440661,55.378706,14.178997,42.917834,23.212199,26.417654,...,64.519703,4.775083,43.981373,72.999857,27.102261,20.996296,3.607103,39.967449,45.141392,addicted
4,51.900094,54.809999,113.999959,336.252881,29.900918,53.569130,11.944623,34.313907,17.342946,30.299428,...,96.734399,3.988225,29.985422,53.775181,13.792507,20.360930,2.889787,51.401620,46.543616,addicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7617,56.398217,64.924927,102.732291,88.351802,48.641285,65.179016,36.246569,81.446588,56.383928,53.427282,...,195.318946,11.285102,173.717064,174.860439,88.949160,86.029325,26.312015,214.825811,91.225219,normal
7618,16.385750,19.103398,36.256479,29.794270,14.541851,18.748430,13.719778,28.332358,13.236784,30.672314,...,90.841723,5.113646,49.058404,54.869916,41.159297,29.461572,11.418493,61.084965,53.495500,normal
7619,53.679662,66.320784,87.491928,83.587297,46.040212,55.310880,36.804056,60.752408,44.290746,36.999455,...,90.035788,14.752258,92.281538,105.084474,51.557043,60.772363,21.107574,126.136996,112.691773,normal
7620,92.897973,89.041783,81.298996,88.143716,46.572059,44.647671,28.145790,38.853392,37.093237,47.966618,...,81.954289,11.033426,51.752403,74.348431,33.722013,33.833968,11.305672,74.203678,77.927507,normal


In [22]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Prepare the data
X = merged_beta_a_c.iloc[:, :-1]
y = merged_beta_a_c.iloc[:, -1]

# Encode the target variable if it's categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define individual models
xgb_classifier = XGBClassifier()
lgb_classifier = lgb.LGBMClassifier()
catboost_classifier = CatBoostClassifier(verbose=0)

# Define voting classifier
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('lgb', lgb_classifier),
        ('catboost', catboost_classifier)
    ],
    voting='soft'  # 'hard' for majority voting, 'soft' for weighted probabilities
)

# Define k-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

voting_scores = cross_validate(voting_classifier, X, y_encoded, cv=kf, scoring=scoring)

print("\nVoting Ensemble Metrics:")
print("Accuracy:", np.mean(voting_scores['test_accuracy']))
print("Precision:", np.mean(voting_scores['test_precision']))
print("Recall:", np.mean(voting_scores['test_recall']))
print("F1 Score:", np.mean(voting_scores['test_f1']))


[LightGBM] [Info] Number of positive: 3439, number of negative: 3420
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001764 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501385 -> initscore=0.005540
[LightGBM] [Info] Start training from score 0.005540
[LightGBM] [Info] Number of positive: 3446, number of negative: 3413
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005531 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 6859, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502406 -> initscore=0.009622
[LightGBM